# ReAct Prompting Example

This notebook demonstrates **ReAct (Reason + Act)**, a technique that combines "Chain of Thought" reasoning with "Action" execution.

**The Concept:**
Humans solve problems by interleaving thinking and acting. We think "I need to check the weather," we look it up (Act), we see it's raining (Observe), and then we think "I should take an umbrella" (Reason).
ReAct forces the LLM to follow this exact cycle: **Thought → Action → Observation → Thought...**

**Key Mechanics:**
1.  **Thought**: The model analyzes the current situation.
2.  **Action**: It decides on a specific tool or step to take next (e.g., "Search Wikipedia").
3.  **Observation**: It looks at the result of that action (in this specific demo, we ask the model to *simulate* the observation to show the flow).
4.  **Repeat**: This loop continues until the final answer is found.

In [ ]:
%pip install openai python-dotenv --quiet

### 1. Setup and Authorization

We start by importing the necessary libraries and loading your OpenAI API key.

In [ ]:
from openai import OpenAI
import os
import time
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

# Model configuration - can be overridden via environment variable
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

client = OpenAI(api_key=api_key)
print(f"OpenAI client ready! Using model: {MODEL}")

### 2. The System Prompt (The "Protocol")

We need to enforce a strict format. Unlike standard chat, a ReAct agent must output specific keywords (`Thought:`, `Action:`, `Observation:`) to structure its problem-solving.

**Note on "Simulation":**
In a production system, you would write code to parse the "Action" and actually execute a Python function or API call. For this prompt engineering demo, we instruct the model to **simulate** the observation so you can see the *reasoning trace* in action without needing external tools connected.

In [ ]:
SYSTEM_PROMPT = """
You are a smart agent using the ReAct (Reason + Act) framework.

For every step, you must follow this strict format:
1. Thought: Reason about the current state.
2. Action: Describe the specific action or tool query you would use.
3. Observation: Simulate a realistic result of that action.
4. Repeat this cycle until you have the answer.
5. Final Answer: State the conclusion clearly.

Do not just guess. "Use" the tools (by simulating them) to get information.
"""

### 3. The Interactive ReAct Agent

This loop runs the agent.

**Why use `temperature=0.3`?**
* ReAct requires logic and strict adherence to a formatting protocol.
* A lower temperature helps keep the model focused on the `Thought` -> `Action` structure without drifting into creative storytelling.

In [ ]:
# Initialize history with the ReAct instructions
messages = [{"role": "system", "content": SYSTEM_PROMPT}]

# Maximum number of messages to keep (prevents token limit issues)
MAX_HISTORY = 20

def trim_message_history(messages, max_messages=MAX_HISTORY):
    """Keep system prompt and last N-1 messages to prevent token overflow."""
    if len(messages) > max_messages:
        return [messages[0]] + messages[-(max_messages - 1):]
    return messages

print("ReAct Agent Ready!")
print("Enter a task that requires multiple steps (e.g., 'What is the population of the capital of France?').")
print("Type 'quit' to exit.\n")

while True:
    # 1. Get User Input
    user_input = input("Your task: ")
    
    if user_input.strip().lower() in ["quit", "exit"]:
        print("Goodbye!")
        break
    
    if not user_input.strip():
        print("Please enter a valid task.")
        continue
    
    # 2. Add to History
    messages.append({"role": "user", "content": user_input})
    
    try:
        # 3. Generate the ReAct Trace
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0.3  # Low temp for logical structure
        )
        
        reply = response.choices[0].message.content
        
        # 4. Display the Process
        print(f"\nReasoning Trace:\n{reply}\n")
        print("-" * 60)
        
        # 5. Save Context
        messages.append({"role": "assistant", "content": reply})
        
        # 6. Trim history to prevent token overflow
        messages = trim_message_history(messages)
        
        # 7. Rate limiting to avoid API throttling
        time.sleep(0.2)
        
    except Exception as e:
        print(f"\nError: {e}\n")
        messages.pop()  # Remove the failed user message